# Logica cu predicate (2). Rezoluție
 - Andrei Olaru
 - Tudor Berariu


## Scopul laboratorului

Familiarizarea cu mecanismul rezoluției și cu strategiile de rezoluție.

#### Resurse

Cursul 5 de IA slides 34-44.

### Cerința 1

* din notebook-ul de la Laboratorul 5 faceți Download as &rarr; Python și salvați fișierul ca `Lab05.py`, în acest director.
* adăugați de asemenea în acest director fișierul `Lab05tester.py` (**descărcați din nou** de pe site).

In [1]:
# TODO
from Lab05 import *

>>> Test batch [0]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
Test 7: OK
Test 8: OK
Test 9: OK
Test 10: OK
Test 11: OK
Test 12: OK
Test 13: OK
Test 14: OK
Test 15: OK
Test 16: OK
Test 17: OK
Test 18: OK
Test 19: OK
Test 20: OK
Test 21: OK
Test 22: OK
Test 23: OK
>>>  24 / 24 tests successful.
(SI (SAU NOTP(?x) Q(?x)) T(?y, <built-in function add>[1,2]))
>>> Test batch [1]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
>>>  5 / 5 tests successful.
>>> Test batch [2]
Test 0: OK
Test 1: OK
Test 2: OK
Test 3: OK
Test 4: OK
Test 5: OK
Test 6: OK
>>>  7 / 7 tests successful.
>>> Test batch [3]
Test 0: OK, got <False>
Test 1: OK, got <{x -> A}>
Test 2: OK, got <{x -> A}>
Test 3: OK, got <{x -> A}>
Test 4: OK, got <{x -> A}>
Test 5: OK, got <{x -> <built-in function add>[?Z,5]}>
Test 6: OK, got <{z -> C, x -> A, y -> B}>
Test 7: OK, got <{x -> 2, y -> 3}>
Test 8: OK, got <False>
Test 9: OK, got <False>
>>>  10 / 10 tests successful.


In [2]:
# Testare conectori cu mai multe propoziții și replace_args()
from Lab05tester import *

dummy = make_atom("P")
dummy2 = make_atom("Q", make_var("x"))
L1 = [dummy, dummy, dummy, dummy]
L2 = [dummy2, dummy2, dummy2, dummy2]

f = make_or(*L1)
print_formula(f) # trebuie să dea ceva gen (V P() P() P() P())
test("get_args(f)", L1, globals(), 0)
f2 = replace_args(f, L2)
test("get_args(f)", L1, globals(), 1) # tests f did not change
test("get_args(f2)", L2, globals(), 2) # tests arguments changed ok

(SAU P() P() P() P())
Test 0: OK
Test 1: OK
Test 2: OK


True

In [3]:
from copy import deepcopy

# în această celulă se găsesc câteva funcții utilizate intern

def check_term(T):
    return (is_constant(T) and get_value(T) is not None) or \
            (is_variable(T) and get_name(T) is not None) or \
            (is_function_call(T) and callable(get_head(T)) and not [t for t in get_args(T) if not check_term(t)])
def check_atom(A):
    return is_atom(A) and get_head(A) is not None and not [t for t in get_args(A) if not check_term(t)]
dummy = make_atom("P")
and_name = get_head(make_and(dummy, dummy))
or_name = get_head(make_and(dummy, dummy))
neg_name = get_head(make_neg(dummy))
def check_sentence(S):
    return is_sentence(S) and (
        check_atom(S) or
        ((get_head(S) == and_name or get_head(S) == or_name) and len(get_args(S)) >= 2 and
             not [s for s in get_args(S) if not check_sentence(s)])
        or
        (get_head(S) == neg_name and len(get_args(S)) == 1 and check_sentence(get_args(S)[0]))
        )

def make_statement(conclusion, hypotheses):
    L = list(hypotheses)
    if not L:
        return conclusion
    L = [make_neg(s) for s in L]
    L.append(conclusion)
    return make_or(*L)

def add_statement(kb, conclusion, *hypotheses):
    s = make_statement(conclusion, hypotheses)
    if check_sentence(s) is not None:
        kb.append(s)
        print("Added statement " + print_formula(s, True))
        return True
    print("Sentence does not check out ", result)
    return False

var_no = 0;
def init_KB():
    global var_no
    var_no = 0
    return []

def assign_next_var_name():
    global var_no
    var_no += 1
    return "v" + str(var_no)

def gather_vars(S):
    if is_variable(S):
        return [get_name(S)]
    if has_args(S):
        res = []
        for a in get_args(S):
            res = res + gather_vars(a)
        return res
    return []

def make_unique_var_names(KB):
    for idx in range(len(KB)):
        S = KB[idx]
        varL = gather_vars(S)
        subst = {}
        for var in varL:
            subst[var] = make_var(assign_next_var_name())
        KB[idx] = substitute(S, subst)
            
            
def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))


In [4]:
# KB 1
# based on an example in Artificial Intelligence - A Modern Approach
KB_America = init_KB()
#0 Mr West is a US general
add_statement(KB_America, make_atom("USGeneral", make_const("West")))
#1 General Awesome is also a US general
add_statement(KB_America, make_atom("USGeneral", make_const("General_Awesome")))
#2 General Awesome is Awesome
add_statement(KB_America, make_atom("Awesome", make_const("General_Awesome")))
#3 Nono is an enemy of America
add_statement(KB_America, make_atom("Enemy", make_const("Nono"), make_const("America")))
#4 M1 is a type of missle
add_statement(KB_America, make_atom("Missle", make_const("M1")))
#5 Nono has the M1 missle
add_statement(KB_America, make_atom("Owns", make_const("Nono"), make_const("M1")))

#6 any US general is an American
add_statement(KB_America, make_atom("American", make_var("x")), make_atom("USGeneral", make_var("x")))
#7 any missle is a weapon
add_statement(KB_America, make_atom("Weapon", make_var("x")), make_atom("Missle", make_var("x")))
#8 if anyone owns a missle, it is General West that sold them that missle
add_statement(KB_America, make_atom("Sells", make_const("West"), make_var("y"), make_var("x")), make_atom("Owns", make_var("x"), make_var("y")), make_atom("Missle", make_var("y")))
#9 any American who sells weapons to a hostile is a criminal
add_statement(KB_America, make_atom("Criminal", make_var("x")), make_atom("Weapon", make_var("y")), make_atom("Sells", make_var("x"), make_var("y"), make_var("z")), make_atom("Hostile", make_var("z")), make_atom("American", make_var("x")))
#10 any enemy of America is called a hostile
add_statement(KB_America, make_atom("Hostile", make_var("x")), make_atom("Enemy", make_var("x"), make_const("America")))
#11 America is awesome if at least an American is awesome
add_statement(KB_America, make_atom("Awesome", make_const("America")), make_atom("American", make_var("x")), make_atom("Awesome", make_var("x")))

make_unique_var_names(KB_America)

print_KB(KB_America)


Added statement USGeneral(West)
Added statement USGeneral(General_Awesome)
Added statement Awesome(General_Awesome)
Added statement Enemy(Nono, America)
Added statement Missle(M1)
Added statement Owns(Nono, M1)
Added statement (SAU NOTUSGeneral(?x) American(?x))
Added statement (SAU NOTMissle(?x) Weapon(?x))
Added statement (SAU NOTOwns(?x, ?y) NOTMissle(?y) Sells(West, ?y, ?x))
Added statement (SAU NOTWeapon(?y) NOTSells(?x, ?y, ?z) NOTHostile(?z) NOTAmerican(?x) Criminal(?x))
Added statement (SAU NOTEnemy(?x, America) Hostile(?x))
Added statement (SAU NOTAmerican(?x) NOTAwesome(?x) Awesome(America))
KB now:
			USGeneral(West)
			USGeneral(General_Awesome)
			Awesome(General_Awesome)
			Enemy(Nono, America)
			Missle(M1)
			Owns(Nono, M1)
			(SAU NOTUSGeneral(?v2) American(?v2))
			(SAU NOTMissle(?v4) Weapon(?v4))
			(SAU NOTOwns(?v9, ?v8) NOTMissle(?v8) Sells(West, ?v8, ?v9))
			(SAU NOTWeapon(?v12) NOTSells(?v16, ?v12, ?v14) NOTHostile(?v14) NOTAmerican(?v16) Criminal(?v16))
			(SAU

In [5]:
# KB 2
# din cursul de IA
KB_Faster = init_KB()

def the_greyhound():
    return make_const("Greg")

#0 horses are faster than dogs
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("y")), make_atom("Horse", make_var("x")), make_atom("Dog", make_var("y")))
#1 there is a greyhound that is faster than any rabbit
add_statement(KB_Faster, make_atom("Faster", make_function_call(the_greyhound), make_var("z")), make_atom("Rabbit", make_var("z")))
#2 Harry is a horse
add_statement(KB_Faster, make_atom("Horse", make_const("Harry")))
#3 Ralph is a rabbit
add_statement(KB_Faster, make_atom("Rabbit", make_const("Ralph")))
#4 Greg is a greyhound
add_statement(KB_Faster, make_atom("Greyhound", make_function_call(the_greyhound)))
#5 A greyhound is a dog
add_statement(KB_Faster, make_atom("Dog", make_var("y")), make_atom("Greyhound", make_var("y")))
#6 transitivity
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("z")),
              make_atom("Faster", make_var("x"), make_var("y")), make_atom("Faster", make_var("y"), make_var("z")))

make_unique_var_names(KB_Faster)

print_KB(KB_Faster)


Added statement (SAU NOTHorse(?x) NOTDog(?y) Faster(?x, ?y))
Added statement (SAU NOTRabbit(?z) Faster(<function the_greyhound at 0x7fa2ac5e4378>[], ?z))
Added statement Horse(Harry)
Added statement Rabbit(Ralph)
Added statement Greyhound(<function the_greyhound at 0x7fa2ac5e4378>[])
Added statement (SAU NOTGreyhound(?y) Dog(?y))
Added statement (SAU NOTFaster(?x, ?y) NOTFaster(?y, ?z) Faster(?x, ?z))
KB now:
			(SAU NOTHorse(?v3) NOTDog(?v4) Faster(?v3, ?v4))
			(SAU NOTRabbit(?v6) Faster(<function the_greyhound at 0x7fa2ac5e4378>[], ?v6))
			Horse(Harry)
			Rabbit(Ralph)
			Greyhound(<function the_greyhound at 0x7fa2ac5e4378>[])
			(SAU NOTGreyhound(?v8) Dog(?v8))
			(SAU NOTFaster(?v13, ?v11) NOTFaster(?v11, ?v14) Faster(?v13, ?v14))


In [6]:
KB_test = init_KB()
add_statement(KB_test, make_atom("Q", make_var("x")), make_atom("P", make_var("x")))
add_statement(KB_test, make_atom("P", make_const("A")))

make_unique_var_names(KB_test)
print_KB(KB_test)

Added statement (SAU NOTP(?x) Q(?x))
Added statement P(A)
KB now:
			(SAU NOTP(?v2) Q(?v2))
			P(A)


### Cerința 2

* Implementați funcția `resolves`, care primește două clauze (literali sau disjuncții de literali) și întoarce `False` dacă cele două clauze nu rezolvă, altfel un tuplu care conține literalii care rezolvă, din cele două clauze, și substituția sub care aceștia rezolvă.

In [15]:
def is_positive_literal(L):
    return is_atom(L)
def is_negative_literal(L):
    global neg_name
    return get_head(L) == neg_name and is_positive_literal(get_args(L)[0])
def is_literal(L):
    return is_positive_literal(L) or is_negative_literal(L)

def resolves(C1, C2):
    #print("testing " + print_formula(C1, True) + " and " + print_formula(C2, True))
    
    # întoarce un tuplu (literal-din-C1, literal-din-C2, substituție)
    # unde literal-din-C1 și literal-din-C2 unifică sub substituție
    res = unify(C1, C2)
    if res != False and res != None:
        return res
    
    if is_atom(C1) and is_atom(C2):
        return False
    if not is_literal(C1):
        args1 = get_args(C1)
    else:
        args1 = [C1]

    if not is_literal(C2):
        args2 = get_args(C2)
    else:
        args2 = [C2]
        
        
    for arg1 in args1:
        for arg2 in args2:
            if is_negative_literal(arg1):
                a1 = get_args(arg1)[0]
                if is_positive_literal(arg2):
                    res = unify(a1, arg2)
                    if res != None and res != False:
                        return (arg1, arg2, res)
            else:
                if is_negative_literal(arg2):
                    a2 = get_args(arg2)[0]
                    res = unify(arg1, a2)
                    if res != None and res != False:
                        return (arg1, arg2, res)
    
    return False

# Test!
test_batch(4, globals())

>>> Test batch [4]
Test 0: OK, got <(USGeneral(General_Awesome), NOTUSGeneral(?v2), {v2 -> General_Awesome}>
Test 1: OK, got <(Owns(Nono, M1), NOTOwns(?v9, ?v8), {v9 -> Nono, v8 -> M1}>
Test 2: OK, got <(NOTAmerican(?v16), American(?v2), {v16 -> ?v2}>
Test 3: OK, got <(Enemy(Nono, America), NOTEnemy(?v18, America), {v18 -> Nono}>
Test 4: OK, got <False>
Test 5: OK, got <False>
Test 6: OK, got <{}>
Test 7: OK, got <{}>
>>>  8 / 8 tests successful.


In [16]:
# prints a 5-tuple resolvent representation (see below)
def print_r(R):
    if R is None:
        print("no resolvent")
    else:
        print("resolvent: " + print_formula(R[2], True) + "/" + print_formula(R[3], True) + str(R[4]) \
                        + "\n\t\t in " + print_formula(R[0], True) + "\n\t\t and " + print_formula(R[1], True))


### Cerința 3

* implementați partea lipsă din funcția `solve_problem`, utilizând o strategie de rezoluție la alegere pentru a alege două clauze care rezolvă, și adăugând rezultatul pasului de rezoluție la baza de cunoștințe.

In [47]:
def solve_problem(hypotheses, conclusion):
    KB = hypotheses[:]
    KB = [make_neg(conclusion)] + KB # puteți adăuga și la sfârșit (în funcție de strategie)
    Effort = 30
    
    S = []
    #print_KB(KB)
    
    while Effort > 0:
        Effort -= 1
        
        # Se calculează un rezolvent, ca tuplu (Clauza1, Clauza2, Literal-din-clauza1, Literal-din-clauza2, substituție)
        resolvent = None # TODO
        found = False
        for c1 in KB:
            if found:
                break
            for c2 in KB:
                if c1 != c2:
                    res = resolves(c1, c2)
                    if res != False:
                        if (c1, c2) not in S:
                            resolvent = (c1, c2, res[0], res[1], res[2])
                            found = True
                            break

        print_r(resolvent)
        if resolvent is None:
            print("Failed (effort left " + str(Effort) + ")")
            return False
        
        S = S + [(resolvent[0], resolvent[1])]
        
        # Se calculează noua clauză de adăugat și se adaugă la baza de cunoștințe
        # Clauza trebuie să fie în acest punct o listă de literali
        C = [] # TODO
        if not is_literal(resolvent[0]):
            args1 = get_args(resolvent[0])
        else:
            args1 = [resolvent[0]]
        if not is_literal(resolvent[1]):
            args2 = get_args(resolvent[1])
        else:
            args2 = [resolvent[1]]
            
        for c in args1:
            if c != resolvent[2]:
                subst = substitute(c, resolvent[4])
                if subst != None:
                    c = subst
                #print("add literal")
                #print(c)
                C.append(c)
        for c in args2:
            if c != resolvent[3]:
                subst = substitute(c, resolvent[4])
                if subst != None:
                    c = subst
                #print("add literal")
                #print(c)
                C.append(c)
    
        #KB.remove(resolvent[0])
        #KB.remove(resolvent[1])
        
        # update KB
        if C == []:
            print("Done (effort left " + str(Effort) + ")")
            return True
        if len(C) == 1:
            C = C[0]
        else:
            C = make_or(*C)
        print("Added: " + print_formula(C, True))
        KB = [C] + KB
        
        #print_KB(KB)      
        
#print_KB(KB_test)
#solve_problem(deepcopy(KB_test), make_atom("Q", make_const("A")))

#print_KB(KB_America)
#solve_problem(deepcopy(KB_America), make_atom("Criminal", make_const("West")))

#print_KB(KB_Faster)
solve_problem(deepcopy(KB_Faster), make_atom("Faster", make_const("Harry"), make_const("Ralph")))


resolvent: NOTFaster(Harry, Ralph)/Faster(?v3, ?v4){'v3': ('CONST', 'Harry'), 'v4': ('CONST', 'Ralph')}
		 in NOTFaster(Harry, Ralph)
		 and (SAU NOTHorse(?v3) NOTDog(?v4) Faster(?v3, ?v4))
Added: (SAU NOTHorse(Harry) NOTDog(Ralph))
resolvent: NOTHorse(Harry)/Horse(Harry){}
		 in (SAU NOTHorse(Harry) NOTDog(Ralph))
		 and Horse(Harry)
Added: NOTDog(Ralph)
resolvent: NOTDog(Ralph)/Dog(?v8){'v8': ('CONST', 'Ralph')}
		 in NOTDog(Ralph)
		 and (SAU NOTGreyhound(?v8) Dog(?v8))
Added: NOTGreyhound(Ralph)
resolvent: NOTDog(Ralph)/Dog(?v8){'v8': ('CONST', 'Ralph')}
		 in (SAU NOTHorse(Harry) NOTDog(Ralph))
		 and (SAU NOTGreyhound(?v8) Dog(?v8))
Added: (SAU NOTHorse(Harry) NOTGreyhound(Ralph))
resolvent: NOTHorse(Harry)/Horse(Harry){}
		 in (SAU NOTHorse(Harry) NOTGreyhound(Ralph))
		 and Horse(Harry)
Added: NOTGreyhound(Ralph)
resolvent: NOTFaster(Harry, Ralph)/Faster(?v13, ?v14){'v14': ('CONST', 'Ralph'), 'v13': ('CONST', 'Harry')}
		 in NOTFaster(Harry, Ralph)
		 and (SAU NOTFaster(?v13, ?

True